In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

fake = Faker('pt_BR')

In [2]:
# Configurações
num_rows = 7000  # 7000 linhas iniciais (o dataset será reduzido ao remover os fins de semanas no final do código)
start_date = datetime(2017, 1, 1, 8, 0, 0)
end_date = datetime(2023, 12, 31, 17, 0, 0)
min_os_duration = timedelta(minutes=5)
max_os_duration = timedelta(days=60)

service_type = ("Manutenção Corretiva")

In [3]:
# Hierarquia de Causa, Defeito e Solução
cause_hierarchy = {
    'Problema no Sistema de Alimentação': {
        'Problema na Rede Elétrica': ['Substituição de cabo danificado'],
        'Problema no Cabo de Alimentação': ['Troca de cabo de alimentação'],
        'Desligamento Inesperado': ['Substituição de fusível', 'Troca de bateria do no-break', 'Substituição de fonte']
    },
    'Falha no Circuito Eletrônico': {
        'Curto-Circuito': ['Substituição de Placa-Eletrônica', 'Substituição de componente danificado', 'Equipamento Sem Reparo'],
        'Sobreaquecimento': ['Limpeza interna do equipamento', 'Substituir cooler defeituoso', 'Substituição de componente danificado'],
        'Problema na Placa de Controle': ['Reparo trilhas danificadas', 'Substituição de componente danificado', 'Atualizar firmware da placa'],
        'Sobrecarga Elétrica': ['Adicionar no-break ao equipamento', 'Corrigido problema elétrico'],
        'Componentes Queimados': ['Substituição de componente danificado', 'Equipamento Sem Reparo']
    },
    'Erro de Calibração': {
        'Desvio na Calibração': ['Calibração', 'Substituição de componente danificado', 'Substituição de Placa-Eletrônica', 'Equipamento Sem Reparo'],
        'Falha no Sensor': ['Limpeza interna do equipamento', 'Substituição de componente danificado'],
        'Configuração Incorreta': ['Reset nas configurações do equipamento', 'Treinamento']
    },
    'Defeito Mecânico': {
        'Obstrução Mecânica': ['Removido obstruções físicas', 'Limpeza do equipamento'],
        'Quebra de Peça Mecânica': ['Substituição de peça quebrada', 'Reparo de peça quebrada']
    },
    'Uso Inadequado': {
        'Mau Uso do Equipamento': ['Treinamento', 'Reset nas configurações do equipamento', 'Reparo de peça quebrada', 'Substituição de componente danificado', 'Equipamento Sem Reparo'],
        'Utilização Fora das Especificações': ['Treinamento', 'Reparo de peça quebrada', 'Substituição de componente danificado', 'Equipamento Sem Reparo']
    },
    'Problema no Software': {
        'Falha no Software': ['Substituição de componente danificado', 'Reset nas configurações do equipamento', 'Atualização de software', 'Manutenção Externa', 'Substituição de Placa-Eletrônica'],
        'Erro de Comunicação': ['Correção nos cabos de comunicação', 'Configurar corretamente as interfaces de comunicação', 'Aberto chamado com TI']
    }
}

In [4]:
# Hierarquia de Equipamentos por Setor, Marca e Modelo
sector_equipment_hierarchy = {
    "UTI": {
        "Cardioversor": {
            "InstaMed": {"CardioMaximum": [234567, 345678]},
        },
        "Monitor Multiparametro": {
            "Estrela": {"Efficiente A": [456789, 567890, 678901], "Efficiente B": [789012, 890123, 901234]},
            "ProVida": {"B12": [123456, 234567, 345678], "Pro15": [456789, 567890, 678901]}
        },
        "Ventilador Pulmonar": {
            "Dragon": {"Evito 600": [789012, 890123, 901234], "Savinho 300": [123456, 234567, 345678]},
        },
        "Bomba de Infusão": {
            "BraumBraum": {"InfusoMath": [456789, 567890, 678901]},
            "Sãotronic": {"Icarus": [789012, 890123, 901234]}
        }
    },
    "Centro Cirúrgico": {
        "Cardioversor": {
            "CmaisDrake": {"Vivium": [123456, 654321]}
        },
        "Carrinho de Anestesia": {
            "Dragon": {"Fabio +": [234567, 345678, 456789], "Fabio + XL": [567890, 678901, 789012]},
        },
        "Monitor Multiparametro": {
            "Dragon": {"Infinito Gama": [678901, 789012, 890123], "Infinito Omega XL": [901234, 123456, 234567]},
        },
        "Bisturi Elétrico": {
            "DeltrãoMix": {"BE-3000": [345678, 456789, 567890], "BE-4000": [678901, 789012, 890123]},
            "ValleDoLab": {"Força X": [901234, 123456, 234567], "Força Y": [345678, 456789, 567890]}
        },
        "Bomba de Infusão": {
            "BraumBraum": {"InfusoMath": [678901, 789012, 890123]},
            "Sãotronic": {"Icarus": [901234, 123456, 234567]}
        }
    },
    "Enfermaria": {
        "Cardioversor": {
             "CmaisDrake": {"Vivium": [234567, 345678, 456789]}
        },
        "Oxímetro de Pulso": {
            "Comtech": {"CCM500": [567890, 678901, 789012]},
            "AlphaMedic": {"Senso 10": [890123, 901234, 123456], "Vitro": [901234, 123456, 234567]},
            "ND": {"VC2000": [123456, 234567, 345678]}
        },
        "Incubadora": {
            "Faznem": {"Neonatal 1835": [456789, 567890, 678901], "Visão Avançada 2083": [567890, 678901, 789012]},
        }
    },
    "Neo": {
        "Cardioversor": {
            "InstaMed": {"CardioMaximum": [901234, 123456]},
        },
        "Monitor Multiparametro": {
            "Estrela": {"Efficiente A": [789012, 890123, 901234], "Efficiente B": [123456, 234567, 345678]},
            "ProVida": {"B12": [567890, 678901, 789012], "Pro15": [678901, 789012, 890123]}
        }
    },
    "Centro de Imagens": {
        "Cardioversor": {
            "InstaMed": {"CardioMaximum": [123456, 234567]},
        },
        "Aparelho de Raio-X": {
            "Estrela": {"DuroDiagnostico": [678901, 789012]},
            "Siemais": {"Multiplica B": [901234, 123456]}
        },
        "Tomógrafo": {
            "Siemais": {"Somation Vai": [234567], "Somation Vai.Top": [567890]}
        },
        "Ultrassom": {
            "Estrela": {"ClaraVista": [890123, 901234], "Epic Top": [123456, 234567], "Afinidade 30": [567890, 678901]},
            "G&E": {"Lógica E Portátil": [789012, 890123]}
        },
        "Carrinho de Anestesia": {
            "Dragon": {"Fabio +": [901234, 123456]},
        }
    },
    "Emergência": {
        "Cardioversor": {
            "InstaMed": {"CardioMaximum": [678901, 789012]},
        },
        "Monitor Multiparametro": {
            "Estrela": {"Efficiente A": [890123, 901234, 123456], "Efficiente B": [234567, 345678, 456789]},
            "ProVida": {"B12": [567890, 678901, 789012], "Pro15": [678901, 789012, 890123]}
        },
        "Oxímetro de Pulso": {
            "Comtech": {"CCM500": [901234, 123456, 234567]},
            "AlphaMedic": {"Senso 10": [567890, 678901, 789012], "Vitro": [890123, 901234, 123456]},
            "ND": {"VC2000": [234567, 345678, 456789]}
        },
        "Ventilador Pulmonar": {
            "Dragon": {"Evito 600": [901234, 123456, 234567], "Savinho 300": [234567, 345678, 456789]},
        },
        "Bomba de Infusão": {
            "BraumBraum": {"InfusoMath": [567890, 678901, 789012, 567890, 678901, 789012, 567890, 678901, 789012]},
            "Sãotronic": {"Icarus": [890123, 901234, 123456, 890123, 901234, 123456, 890123, 901234, 123456]}
        }
    },
    "Cardiologia": {
        "Cardioversor": {
            "CmaisDrake": {"Vivium": [901234, 123456]}
        },
        "Oxímetro de Pulso": {
            "Comtech": {"CCM500": [234567, 345678, 456789]},
            "AlphaMedic": {"Senso 10": [567890, 678901, 789012], "Vitro": [789012, 890123, 901234]},
            "ND": {"VC2000": [901234, 123456, 234567]}
        }
    }
    # Adicione os demais setores conforme necessário
}


In [5]:
# Adicione a hierarquia de criticidade apenas pelo equipamento
equipment_criticality_hierarchy = {
    "Cardioversor": "Médio",
    "Monitor Multiparametro": "Médio",
    "Ventilador Pulmonar": "Médio",
    "Bomba de Infusão": "Baixo",
    "Oxímetro de Pulso": "Baixo",
    "Ultrassom": "Médio",
    "Aparelho de Raio-X": "Alto",
    "Carrinho de Anestesia": "Alto",
    "Tomógrafo": "Alto",
    "Incubadora": "Médio",
    "Bisturi Elétrico": "Médio",
    
    # Adicione outras categorias conforme necessário
}

In [6]:
# Função para gerar dados de O.S.
def generate_os_data_row(i):
    
    # Verificar se há setores disponíveis
    if not sector_equipment_hierarchy:
        return None  # Ou faça algo apropriado para lidar com o caso em que não há setores

    # Selecionar um setor aleatório
    sector = random.choice(list(sector_equipment_hierarchy.keys()))

    sector_hierarchy = sector_equipment_hierarchy.get(sector, {})

    # Verificar se o setor possui equipamentos antes de prosseguir
    if not sector_hierarchy:
        return None  # Ou faça algo apropriado para lidar com o caso em que não há equipamentos no setor

    equipment = random.choice(list(sector_hierarchy.keys()))

    # Verificar se o equipamento possui marcas antes de prosseguir
    brand_dict = sector_hierarchy.get(equipment, {})

    if not brand_dict or not isinstance(brand_dict, dict):
        return None  # Ou faça algo apropriado para lidar com o caso em que não há marcas para o equipamento

    brand = random.choice(list(brand_dict.keys()))

    # Verificar se a marca possui modelos antes de prosseguir
    model_dict = brand_dict.get(brand, {})

    if not model_dict or not isinstance(model_dict, dict):
        return None  # Ou faça algo apropriado para lidar com o caso em que não há modelos para a marca

    model = random.choice(list(model_dict.keys()))

    # Verificar se o modelo possui N/S antes de prosseguir
    serial_numbers = model_dict.get(model, [])

    if not serial_numbers or not isinstance(serial_numbers, list):
        return None  # Ou faça algo apropriado para lidar com o caso em que não há N/S para o modelo

    serial_number = random.choice(serial_numbers)
    technician = random.choice(["Denys", "Huende", "Luís", "Thalita"])

    # Selecionar uma causa aleatória
    cause = random.choice(list(cause_hierarchy.keys()))

    # Selecionar um defeito e uma solução aleatórios associados à causa
    cause_defect_hierarchy = cause_hierarchy[cause]
    defect = random.choice(list(cause_defect_hierarchy.keys()))
    solution = random.choice(cause_defect_hierarchy[defect])

    # Calcular datas respeitando horário de trabalho e dias úteis
    days_before_end = num_rows - i
    weight = i / num_rows  # Ajuste de peso para balancear as datas
    adjusted_start_date = start_date + timedelta(days=int(weight * (end_date - start_date).days))

    # Introduzir uma variabilidade na quantidade de dias entre 0 e 10
    days_variation = random.randint(0, 10)

    adjusted_start_date = start_date + timedelta(days=int(weight * (end_date - start_date).days)) + timedelta(days=days_variation)

    open_date = fake.date_time_between(start_date=adjusted_start_date, end_date=end_date - timedelta(days=days_before_end), tzinfo=None)

    # Ajustar a hora de abertura para um horário aleatório entre 08h e 17h
    open_date = open_date.replace(hour=random.randint(8, 16), minute=random.randint(0, 59), second=random.randint(0, 59))

    # Calcular a hora de fechamento respeitando o horário comercial
    close_date = open_date + min_os_duration + timedelta(minutes=random.randint(0, 420))  # Adiciona até 7 horas

    # Limitar a duração da O.S. a 60 dias
    if close_date - open_date > max_os_duration:
        close_date = open_date + max_os_duration

    # Verificar se a O.S. fechará fora do horário comercial
    if close_date.hour > 17:
        # Ajustar a hora de fechamento para o dia útil seguinte
        close_date = close_date.replace(hour=8, minute=0, second=0) + timedelta(days=1)

    return [open_date.strftime('%Y-%m-%d %H:%M:%S'), sector, equipment, brand, model, serial_number, service_type, technician, cause, defect, solution, close_date.strftime('%Y-%m-%d %H:%M:%S')]

In [7]:
# Criar DataFrame usando multithreading
with ThreadPoolExecutor() as executor:
    os_data = list(executor.map(generate_os_data_row, range(1, num_rows + 1)))
# Garantir que os_data seja uma lista de listas
os_data = [row for row in os_data if row is not None]

# Criar DataFrame
columns = ['Data_de_abertura_da_OS', 'Setor', 'Equipamento', 'Marca', 'Modelo', 'N/S', 'Tipo_de_Serviço', 'Técnico', 'Causa', 'Defeito', 'Solução', 'Data_de_Fechamento_da_OS']
os_df = pd.DataFrame(os_data, columns=columns)
# Remover entradas referentes a finais de semana
os_df['Data_de_abertura_da_OS'] = pd.to_datetime(os_df['Data_de_abertura_da_OS'])
os_df = os_df[os_df['Data_de_abertura_da_OS'].dt.dayofweek < 5]  # Filtrar apenas dias de semana
# Ordenar DataFrame por Data de Abertura
os_df = os_df.sort_values(by='Data_de_abertura_da_OS')

# Resetar índices após ordenação
os_df.reset_index(drop=True, inplace=True)

# Adicionar a coluna de criticidade ao DataFrame
os_df['Criticidade'] = os_df['Equipamento'].map(equipment_criticality_hierarchy)


In [8]:
# Ajuste na função para ajustar o horário para o horário comercial
def adjust_to_business_hours(dt):
    # Se a data estiver no fim de semana, ajusta para a próxima segunda-feira
    if dt.weekday() >= 5:  # 5 e 6 representam sábado e domingo
        next_business_day = pd.offsets.BDay().rollback(dt + pd.Timedelta(days=1))
        dt = next_business_day.replace(hour=8, minute=5, second=0)

    # Define o horário comercial
    business_hours = pd.date_range(start=dt.replace(hour=8, minute=5, second=0),
                                   end=dt.replace(hour=16, minute=45, second=0), freq='T')

    # Garante que a data esteja dentro do horário comercial
    dt = min(business_hours, key=lambda x: abs((x - dt).total_seconds()))
    return dt

# Ajustar a função para calcular Inicio_Acao, Fim_Acao e Data_de_Fechamento_da_OS
def calculate_start_end_times(row):
    criticidade = row['Criticidade']

    if criticidade == 'Alto':
        inicio_limite = (5, 30)
        fim_limite = (10, 43200)
        data_fechamento_limite = (10, 20)
    elif criticidade == 'Médio':
        inicio_limite = (5, 120)
        fim_limite = (10, 43200)
        data_fechamento_limite = (10, 60)
    elif criticidade == 'Baixo':
        inicio_limite = (5, 240)
        fim_limite = (10, 43200)
        data_fechamento_limite = (10, 120)
    else:
        return pd.Series([None, None, None])

    # Calcular Inicio_Acao
    inicio_aleatorio = np.random.randint(*inicio_limite)
    inicio_acao = pd.to_datetime(row['Data_de_abertura_da_OS']) + pd.Timedelta(minutes=inicio_aleatorio)
    inicio_acao = adjust_to_business_hours(inicio_acao)

    # Calcular Fim_Acao
    fim_aleatorio = np.random.randint(*fim_limite)
    fim_acao = inicio_acao + pd.Timedelta(minutes=fim_aleatorio)
    fim_acao = adjust_to_business_hours(fim_acao)

    # Calcular Data_de_Fechamento_da_OS
    data_fechamento_aleatoria = np.random.randint(*data_fechamento_limite)
    data_fechamento = fim_acao + pd.Timedelta(minutes=data_fechamento_aleatoria)
    data_fechamento = adjust_to_business_hours(data_fechamento)

    return pd.Series([inicio_acao, fim_acao, data_fechamento])

In [9]:
# Aplicação da função para adicionar as colunas
os_df[['Inicio_Acao', 'Fim_Acao', 'Data_de_Fechamento_da_OS']] = os_df.apply(calculate_start_end_times, axis=1)

# Ordenação do DataFrame
os_df = os_df.sort_values(by=['Data_de_abertura_da_OS', 'Inicio_Acao', 'Fim_Acao', 'Data_de_Fechamento_da_OS'])

# Adicionar Numero_da_OS ao Dataframe
os_df['Numero_da_OS'] = range(1, len(os_df) + 1)

# Reorganização do DataFrame
os_df = os_df[['Data_de_abertura_da_OS', 'Inicio_Acao', 'Numero_da_OS', 'Setor', 'Equipamento', 'Marca', 'Modelo', 'N/S', 'Criticidade', 'Tipo_de_Serviço', 'Técnico', 'Causa', 'Defeito', 'Solução', 'Fim_Acao', 'Data_de_Fechamento_da_OS']]

# Salvar DataFrame em um arquivo CSV
os_df.to_csv('dataset_engenharia_clinica1.csv', index=False)

/tmp/ipykernel_1548783/1117788393.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  business_hours = pd.date_range(start=dt.replace(hour=8, minute=5, second=0),
